In [1]:
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

In [ ]:
!git clone https://github.com/JeffSackmann/tennis_atp